In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import japanize_matplotlib
from itertools import product
from sklearn.model_selection import ParameterGrid
from typing import Dict, Tuple, List

from horgues3.dataset import HorguesDataset
from horgues3.models import HorguesModel
from horgues3.pluckett_luce import PluckettLuceKeibaBetting
from horgues3.odds import get_odds_dataframes
from torch.utils.data import DataLoader

In [ ]:
# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# デバイス設定
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
logger.info(f"使用デバイス: {device}")

In [ ]:
# 評価パラメータ設定
BATCH_SIZE = 32
NUM_HORSES = 18
HORSE_HISTORY_LENGTH = 18
HISTORY_DAYS = 365
EXCLUDE_HOURS_BEFORE_RACE = 2

# 評価期間設定
EVAL_START_DATE = '20240101'
EVAL_END_DATE = '20241231'

logger.info(f"評価期間: {EVAL_START_DATE} - {EVAL_END_DATE}")

In [ ]:
# 学習済みモデルのロード
logger.info("学習済みモデルをロード中...")
checkpoint = torch.load('outputs/best_model.pth', map_location=device)

# モデル設定の取得
model_config = checkpoint['model_config']
preprocessing_params = checkpoint['preprocessing_params']

logger.info(f"保存されたモデル - Ecpoch: {checkpoint['epoch']+1}")
logger.info(f"学習時検証損失: {checkpoint['val_loss']:.4f}")

In [ ]:
# モデルの再構築
model = HorguesModel(
    sequence_names=model_config['sequence_names'],
    feature_aliases=model_config['feature_aliases'],
    numerical_features=model_config['numerical_features'],
    categorical_features=model_config['categorical_features'],
    d_token=256,
    num_bins=10,
    binning_temperature=1.0,
    binning_init_range=3.0,
    ft_n_layers=3,
    ft_n_heads=8,
    ft_d_ffn=1024,
    seq_n_layers=3,
    seq_n_heads=8,
    seq_d_ffn=1024,
    race_n_layers=3,
    race_n_heads=8,
    race_d_ffn=1024,
    dropout=0.1
).to(device)

# 学習済み重みのロード
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

logger.info("モデルのロードが完了しました")

In [ ]:
# 評価用データセットの作成
logger.info("評価データセットを作成中...")
eval_dataset = HorguesDataset(
    start_date=EVAL_START_DATE,
    end_date=EVAL_END_DATE,
    num_horses=NUM_HORSES,
    horse_history_length=HORSE_HISTORY_LENGTH,
    history_days=HISTORY_DAYS,
    exclude_hours_before_race=EXCLUDE_HOURS_BEFORE_RACE,
    preprocessing_params=preprocessing_params,
    cache_dir='cache/eval',
    use_cache=True
)

eval_loader = DataLoader(
    eval_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

logger.info(f"評価データサイズ: {len(eval_dataset)}")

In [ ]:
# Plackett-Luceモデルの初期化
pl_model = PluckettLuceKeibaBetting(num_horses=NUM_HORSES).to(device)

In [ ]:
# 予測結果を保存するリスト
all_predictions = []
all_rankings = []
all_race_ids = []
all_masks = []
all_probabilities = []

logger.info("モデル予測を実行中...")

with torch.no_grad():
    for batch in tqdm(eval_loader, desc="予測実行中"):
        # データをデバイスに移動
        x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
        x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
        sequence_data = {}
        for seq_name, seq_data in batch['sequence_data'].items():
            sequence_data[seq_name] = {
                'x_num': {k: v.to(device) for k, v in seq_data['x_num'].items()},
                'x_cat': {k: v.to(device) for k, v in seq_data['x_cat'].items()},
                'mask': seq_data['mask'].to(device)
            }
        mask = batch['mask'].to(device)
        rankings = batch['rankings'].to(device)
        race_ids = batch['race_id']
        
        # モデル予測
        scores = model(x_num, x_cat, sequence_data, mask)
        
        # 出走頭数を計算
        num_horses_running = mask.sum(dim=1)
        
        # Plackett-Luceで各馬券種の確率を計算
        probabilities = pl_model(scores, num_horses_running)
        
        # 結果を保存
        all_predictions.append(scores.cpu())
        all_rankings.append(rankings.cpu())
        all_race_ids.extend(race_ids)
        all_masks.append(mask.cpu())
        all_probabilities.append({k: v.cpu() for k, v in probabilities.items()})

# リストを結合
all_predictions = torch.cat(all_predictions, dim=0)
all_rankings = torch.cat(all_rankings, dim=0)
all_masks = torch.cat(all_masks, dim=0)

# 確率データを結合
combined_probabilities = {}
for bet_type in all_probabilities[0].keys():
    combined_probabilities[bet_type] = torch.cat([prob[bet_type] for prob in all_probabilities], dim=0)

logger.info("予測完了")

In [ ]:
# 予測精度の評価関数
def evaluate_ranking_accuracy(predictions, rankings, masks):
    """順位予測精度を評価"""
    results = {}
    
    # 有効なレースのみを抽出
    valid_races = []
    for i in range(len(predictions)):
        if masks[i].sum() >= 3:  # 最低3頭以上が必要
            race_mask = masks[i].bool()
            race_pred = predictions[i][race_mask]
            race_rank = rankings[i][race_mask]
            
            # 順位が付いている馬のみを対象
            ranked_mask = race_rank > 0
            if ranked_mask.sum() >= 3:  # 最低3頭の順位が必要
                valid_races.append({
                    'predictions': race_pred[ranked_mask],
                    'rankings': race_rank[ranked_mask],
                    'race_idx': i
                })
    
    logger.info(f"有効レース数: {len(valid_races)}")
    
    # 1位的中率
    top1_correct = 0
    top3_correct = 0
    
    # 順位相関
    rank_correlations = []
    
    for race in valid_races:
        pred_scores = race['predictions']
        true_ranks = race['rankings']
        
        # 予測順位（スコアの降順）
        pred_ranks = torch.argsort(pred_scores, descending=True) + 1
        
        # 実際の1位馬
        true_winner = torch.argmin(true_ranks)  # 順位1が最小値
        
        # 予測1位馬
        pred_winner = torch.argmax(pred_scores)
        
        # 1位的中
        if true_winner == pred_winner:
            top1_correct += 1
        
        # 3位以内的中（予測1位が実際3位以内）
        if true_ranks[pred_winner] <= 3:
            top3_correct += 1
        
        # スピアマン順位相関
        try:
            # 順位の逆順（1位が最高）に変換してから相関計算
            true_ranks_inv = len(true_ranks) + 1 - true_ranks
            pred_ranks_inv = len(pred_ranks) + 1 - pred_ranks.float()
            
            correlation = torch.corrcoef(torch.stack([true_ranks_inv.float(), pred_ranks_inv]))[0, 1]
            if not torch.isnan(correlation):
                rank_correlations.append(correlation.item())
        except:
            pass
    
    results['win_accuracy'] = top1_correct / len(valid_races) if valid_races else 0
    results['top3_accuracy'] = top3_correct / len(valid_races) if valid_races else 0
    results['rank_correlation'] = np.mean(rank_correlations) if rank_correlations else 0
    results['valid_races'] = len(valid_races)
    
    return results

# 精度評価実行
accuracy_results = evaluate_ranking_accuracy(all_predictions, all_rankings, all_masks)

logger.info("=== 予測精度評価結果 ===")
logger.info(f"1位的中率: {accuracy_results['win_accuracy']:.3f}")
logger.info(f"3位以内的中率: {accuracy_results['top3_accuracy']:.3f}")
logger.info(f"順位相関係数: {accuracy_results['rank_correlation']:.3f}")
logger.info(f"有効レース数: {accuracy_results['valid_races']}")

In [ ]:
# オッズデータの取得
logger.info("オッズデータを取得中...")
try:
    odds_data = get_odds_dataframes(
        start_date=EVAL_START_DATE,
        end_date=EVAL_END_DATE,
        num_horses=NUM_HORSES
    )
    logger.info(f"オッズデータ取得完了: {len(odds_data)}種類")
except Exception as e:
    logger.error(f"オッズデータ取得エラー: {e}")
    odds_data = {}

In [ ]:
# 払戻データの取得
logger.info("払戻データを取得中...")
try:
    from horgues3.haraimodoshi import get_haraimodoshi_dataframes
    
    haraimodoshi_data = get_haraimodoshi_dataframes(
        start_date=EVAL_START_DATE,
        end_date=EVAL_END_DATE,
        num_horses=NUM_HORSES
    )
    logger.info(f"払戻データ取得完了: {len(haraimodoshi_data)}種類")
except Exception as e:
    logger.error(f"払戻データ取得エラー: {e}")
    haraimodoshi_data = {}

In [ ]:
def calculate_expected_value_and_simulate_betting(
    probabilities: torch.Tensor, 
    odds: pd.DataFrame, 
    haraimodoshi: pd.DataFrame,
    race_ids: List[str],
    alpha: float, 
    beta: float,
    bet_type: str
) -> Dict[str, float]:
    """
    期待値ベースの購入判断とシミュレーション
    
    Args:
        probabilities: 予測的中確率 (num_races, num_combinations)
        odds: オッズデータ (num_races, num_combinations)
        haraimodoshi: 払戻データ (num_races, num_combinations)
        race_ids: レースID一覧
        alpha: 期待値閾値
        beta: 確率閾値
        bet_type: 馬券種名
    
    Returns:
        シミュレーション結果
    """
    total_bet = 0
    total_return = 0
    num_bets = 0
    win_count = 0
    
    results = []
    
    # レースごとに処理
    for i, race_id in enumerate(race_ids):
        race_probs = probabilities[i].cpu().numpy()
        
        # オッズとハライモドシの取得
        if race_id not in odds.index or race_id not in haraimodoshi.index:
            continue
            
        race_odds = odds.loc[race_id].values
        race_haraimodoshi = haraimodoshi.loc[race_id].values
        
        # 期待値計算: 期待値 = 予測的中確率 × オッズ - 1
        expected_values = race_probs * race_odds - 1
        
        # 購入判断: 期待値 > alpha AND 予測的中確率 > beta
        buy_mask = (expected_values > alpha) & (race_probs > beta) & (race_odds > 0)
        
        if not buy_mask.any():
            continue
            
        # 購入対象の組み合わせ
        buy_indices = np.where(buy_mask)[0]
        
        for buy_idx in buy_indices:
            # 各馬券に100円ずつ購入
            bet_amount = 100
            total_bet += bet_amount
            num_bets += 1
            
            # 的中判定: 払戻金額が0より大きければ的中
            is_hit = race_haraimodoshi[buy_idx] > 0
            
            if is_hit:
                # 実際の払戻金額を使用
                return_amount = race_haraimodoshi[buy_idx]
                total_return += return_amount
                win_count += 1
            else:
                return_amount = 0
            
            results.append({
                'race_id': race_id,
                'bet_type': bet_type,
                'combination_idx': buy_idx,
                'probability': race_probs[buy_idx],
                'odds': race_odds[buy_idx],
                'expected_value': expected_values[buy_idx],
                'bet_amount': bet_amount,
                'is_hit': is_hit,
                'return_amount': return_amount
            })
    
    # 評価指標計算
    roi = (total_return - total_bet) / total_bet if total_bet > 0 else 0
    hit_rate = win_count / num_bets if num_bets > 0 else 0
    
    return {
        'total_bet': total_bet,
        'total_return': total_return,
        'profit': total_return - total_bet,
        'roi': roi,
        'hit_rate': hit_rate,
        'num_bets': num_bets,
        'win_count': win_count,
        'results': results
    } 

In [ ]:
def optimize_parameters_grid_search(
    probabilities: Dict[str, torch.Tensor],
    odds_data: Dict[str, pd.DataFrame],
    haraimodoshi_data: Dict[str, pd.DataFrame],
    race_ids: List[str],
    alpha_range: np.ndarray,
    beta_range: np.ndarray,
    train_ratio: float = 0.5
) -> Dict[str, Dict[str, float]]:
    """
    グリッドサーチによるパラメータ最適化
    
    Args:
        probabilities: 各馬券種の予測確率
        odds_data: 各馬券種のオッズデータ
        haraimodoshi_data: 各馬券種の払戻データ
        race_ids: レースID一覧
        alpha_range: 期待値閾値の候補
        beta_range: 確率閾値の候補
        train_ratio: 学習データの割合
    
    Returns:
        各馬券種の最適パラメータと結果
    """
    
    # データ分割
    n_races = len(race_ids)
    split_idx = int(n_races * train_ratio)
    
    train_race_ids = race_ids[:split_idx]
    test_race_ids = race_ids[split_idx:]
    
    logger.info(f"学習データ: {len(train_race_ids)}レース")
    logger.info(f"テストデータ: {len(test_race_ids)}レース")
    
    results = {}
    
    for bet_type in ['tansho', 'fukusho', 'umaren', 'wide', 'umatan', 'sanrenpuku', 'sanrentan']:
        if bet_type not in probabilities or bet_type not in odds_data or bet_type not in haraimodoshi_data:
            logger.warning(f"{bet_type}のデータが見つかりません")
            continue
            
        logger.info(f"\n=== {bet_type} パラメータ最適化 ===")
        
        bet_probs = probabilities[bet_type]
        bet_odds = odds_data[bet_type]
        bet_haraimodoshi = haraimodoshi_data[bet_type]
        
        # 学習データでの最適化
        best_roi = -np.inf
        best_params = None
        best_train_result = None
        
        param_results = []
        
        for alpha, beta in product(alpha_range, beta_range):
            # 学習データでシミュレーション
            train_result = calculate_expected_value_and_simulate_betting(
                bet_probs[:split_idx], 
                bet_odds, 
                bet_haraimodoshi,
                train_race_ids,
                alpha, 
                beta, 
                bet_type
            )
            
            # ROIで評価（購入回数が少なすぎる場合は除外）
            if train_result['num_bets'] >= 10:
                param_results.append({
                    'alpha': alpha,
                    'beta': beta,
                    'train_roi': train_result['roi'],
                    'train_hit_rate': train_result['hit_rate'],
                    'train_num_bets': train_result['num_bets']
                })
                
                if train_result['roi'] > best_roi:
                    best_roi = train_result['roi']
                    best_params = (alpha, beta)
                    best_train_result = train_result
        
        if best_params is None:
            logger.warning(f"{bet_type}: 最適パラメータが見つかりませんでした")
            continue
        
        # テストデータで検証
        test_result = calculate_expected_value_and_simulate_betting(
            bet_probs[split_idx:], 
            bet_odds, 
            bet_haraimodoshi,
            test_race_ids,
            best_params[0], 
            best_params[1], 
            bet_type
        )
        
        results[bet_type] = {
            'best_alpha': best_params[0],
            'best_beta': best_params[1],
            'train_roi': best_train_result['roi'],
            'train_hit_rate': best_train_result['hit_rate'],
            'train_num_bets': best_train_result['num_bets'],
            'test_roi': test_result['roi'],
            'test_hit_rate': test_result['hit_rate'],
            'test_num_bets': test_result['num_bets'],
            'test_profit': test_result['profit'],
            'param_results': param_results
        }
        
        logger.info(f"最適パラメータ: α={best_params[0]:.3f}, β={best_params[1]:.3f}")
        logger.info(f"学習ROI: {best_train_result['roi']:.3f}, 的中率: {best_train_result['hit_rate']:.3f}")
        logger.info(f"テストROI: {test_result['roi']:.3f}, 的中率: {test_result['hit_rate']:.3f}")
        logger.info(f"テスト購入回数: {test_result['num_bets']}, 収益: {test_result['profit']:+.0f}円")
    
    return results

In [ ]:
# 対数分布でパラメータ候補を生成
logger.info("パラメータ候補を生成中...")

# α (期待値閾値): 0.001 ~ 0.5 の対数分布
alpha_candidates = np.logspace(-3, np.log10(0.5), 15)

# β (確率閾値): 0.001 ~ 0.5 の対数分布  
beta_candidates = np.logspace(-3, np.log10(0.5), 15)

logger.info(f"α候補数: {len(alpha_candidates)}")
logger.info(f"β候補数: {len(beta_candidates)}")
logger.info(f"総組み合わせ数: {len(alpha_candidates) * len(beta_candidates)}")
logger.info(f"α範囲: {alpha_candidates[0]:.4f} ~ {alpha_candidates[-1]:.4f}")
logger.info(f"β範囲: {beta_candidates[0]:.4f} ~ {beta_candidates[-1]:.4f}")

In [ ]:
# パラメータ最適化の実行（修正版）
logger.info("期待値ベースパラメータ最適化を開始...")

# レースIDリストの作成（評価データから）
race_id_set = set(all_race_ids)
valid_race_ids = []

# オッズデータと払戻データが両方あるレースのみ抽出
for bet_type in ['tansho', 'fukusho', 'umaren', 'wide', 'umatan', 'sanrenpuku', 'sanrentan']:
    if bet_type in odds_data and bet_type in haraimodoshi_data:
        race_id_set = race_id_set.intersection(set(odds_data[bet_type].index))
        race_id_set = race_id_set.intersection(set(haraimodoshi_data[bet_type].index))

valid_race_ids = sorted(list(race_id_set))
logger.info(f"オッズ・払戻データが揃っているレース数: {len(valid_race_ids)}")

# 最適化実行
optimization_results = optimize_parameters_grid_search(
    probabilities=combined_probabilities,
    odds_data=odds_data,
    haraimodoshi_data=haraimodoshi_data,
    race_ids=valid_race_ids,
    alpha_range=alpha_candidates,
    beta_range=beta_candidates,
    train_ratio=0.5
)

In [ ]:
# 結果の可視化
def plot_optimization_results(results: Dict[str, Dict], save_fig: bool = True):
    """最適化結果の可視化"""
    
    bet_types = list(results.keys())
    if not bet_types:
        logger.warning("可視化する結果がありません")
        return
    
    # サブプロット作成
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()
    
    for i, bet_type in enumerate(bet_types):
        if i >= 8:  # 最大8種類まで
            break
            
        result = results[bet_type]
        param_results = result['param_results']
        
        if not param_results:
            continue
            
        # データ準備
        df = pd.DataFrame(param_results)
        
        # ヒートマップ用にピボット
        pivot_data = df.pivot(index='beta', columns='alpha', values='train_roi')
        
        # ヒートマップ描画
        im = axes[i].imshow(pivot_data.values, aspect='auto', cmap='RdYlBu_r')
        axes[i].set_title(f'{bet_type.upper()}\n最適: α={result["best_alpha"]:.3f}, β={result["best_beta"]:.3f}')
        axes[i].set_xlabel('α (期待値閾値)')
        axes[i].set_ylabel('β (確率閾値)')
        
        # 軸ラベル設定
        alpha_labels = [f'{x:.3f}' for x in pivot_data.columns[::3]]
        beta_labels = [f'{x:.3f}' for x in pivot_data.index[::3]]
        axes[i].set_xticks(range(0, len(pivot_data.columns), 3))
        axes[i].set_xticklabels(alpha_labels, rotation=45)
        axes[i].set_yticks(range(0, len(pivot_data.index), 3))
        axes[i].set_yticklabels(beta_labels)
        
        # カラーバー
        plt.colorbar(im, ax=axes[i], label='ROI')
    
    # 余分なサブプロットを非表示
    for j in range(len(bet_types), len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    if save_fig:
        plt.savefig('outputs/parameter_optimization_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()

# 結果可視化
if optimization_results:
    plot_optimization_results(optimization_results)

In [ ]:
# 結果サマリーの表示
def display_optimization_summary(results: Dict[str, Dict]):
    """最適化結果のサマリー表示"""
    
    summary_data = []
    
    for bet_type, result in results.items():
        summary_data.append({
            '馬券種': bet_type.upper(),
            '最適α': f"{result['best_alpha']:.3f}",
            '最適β': f"{result['best_beta']:.3f}",
            '学習ROI': f"{result['train_roi']:.3f}",
            '学習的中率': f"{result['train_hit_rate']:.3f}",
            '学習購入回数': result['train_num_bets'],
            'テストROI': f"{result['test_roi']:.3f}",
            'テスト的中率': f"{result['test_hit_rate']:.3f}",
            'テスト購入回数': result['test_num_bets'],
            'テスト収益': f"{result['test_profit']:+.0f}円"
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    logger.info("\n=== パラメータ最適化結果サマリー ===")
    print(summary_df.to_string(index=False))
    
    # CSVで保存
    summary_df.to_csv('outputs/optimization_summary.csv', index=False, encoding='utf-8-sig')
    logger.info("結果をoutputs/optimization_summary.csvに保存しました")

# サマリー表示
if optimization_results:
    display_optimization_summary(optimization_results)

In [ ]:
# より詳細な分析: 馬券種別のROI分布
def analyze_bet_type_performance(results: Dict[str, Dict]):
    """馬券種別パフォーマンス分析"""
    
    plt.figure(figsize=(15, 10))
    
    # ROI比較
    plt.subplot(2, 2, 1)
    bet_types = list(results.keys())
    train_rois = [results[bt]['train_roi'] for bt in bet_types]
    test_rois = [results[bt]['test_roi'] for bt in bet_types]
    
    x = np.arange(len(bet_types))
    width = 0.35
    
    plt.bar(x - width/2, train_rois, width, label='学習', alpha=0.8)
    plt.bar(x + width/2, test_rois, width, label='テスト', alpha=0.8)
    plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    plt.xlabel('馬券種')
    plt.ylabel('ROI')
    plt.title('馬券種別ROI比較')
    plt.xticks(x, [bt.upper() for bt in bet_types], rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 的中率比較
    plt.subplot(2, 2, 2)
    train_hits = [results[bt]['train_hit_rate'] for bt in bet_types]
    test_hits = [results[bt]['test_hit_rate'] for bt in bet_types]
    
    plt.bar(x - width/2, train_hits, width, label='学習', alpha=0.8)
    plt.bar(x + width/2, test_hits, width, label='テスト', alpha=0.8)
    plt.xlabel('馬券種')
    plt.ylabel('的中率')
    plt.title('馬券種別的中率比較')
    plt.xticks(x, [bt.upper() for bt in bet_types], rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 購入回数比較
    plt.subplot(2, 2, 3)
    train_bets = [results[bt]['train_num_bets'] for bt in bet_types]
    test_bets = [results[bt]['test_num_bets'] for bt in bet_types]
    
    plt.bar(x - width/2, train_bets, width, label='学習', alpha=0.8)
    plt.bar(x + width/2, test_bets, width, label='テスト', alpha=0.8)
    plt.xlabel('馬券種')
    plt.ylabel('購入回数')
    plt.title('馬券種別購入回数比較')
    plt.xticks(x, [bt.upper() for bt in bet_types], rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 収益比較（テストデータのみ）
    plt.subplot(2, 2, 4)
    test_profits = [results[bt]['test_profit'] for bt in bet_types]
    colors = ['green' if p > 0 else 'red' for p in test_profits]
    
    plt.bar(bet_types, test_profits, color=colors, alpha=0.7)
    plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    plt.xlabel('馬券種')
    plt.ylabel('収益（円）')
    plt.title('馬券種別テスト収益')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # 値をバーの上に表示
    for i, v in enumerate(test_profits):
        plt.text(i, v + (max(test_profits) * 0.01), f'{v:+.0f}', 
                ha='center', va='bottom' if v > 0 else 'top')
    
    plt.tight_layout()
    plt.savefig('outputs/bet_type_performance_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

# パフォーマンス分析実行
if optimization_results:
    analyze_bet_type_performance(optimization_results)

In [ ]:
# 最適パラメータの分析
def analyze_optimal_parameters(results: Dict[str, Dict]):
    """最適パラメータの傾向分析"""
    
    plt.figure(figsize=(12, 8))
    
    bet_types = list(results.keys())
    alphas = [results[bt]['best_alpha'] for bt in bet_types]
    betas = [results[bt]['best_beta'] for bt in bet_types]
    test_rois = [results[bt]['test_roi'] for bt in bet_types]
    
    # パラメータの散布図
    plt.subplot(2, 2, 1)
    scatter = plt.scatter(alphas, betas, c=test_rois, s=100, cmap='RdYlGn', alpha=0.7)
    plt.xlabel('α (期待値閾値)')
    plt.ylabel('β (確率閾値)')
    plt.title('最適パラメータの分布')
    plt.colorbar(scatter, label='テストROI')
    
    # 馬券種でラベル付け
    for i, bt in enumerate(bet_types):
        plt.annotate(bt.upper(), (alphas[i], betas[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # αの分布
    plt.subplot(2, 2, 2)
    plt.hist(alphas, bins=10, alpha=0.7, edgecolor='black')
    plt.xlabel('α (期待値閾値)')
    plt.ylabel('頻度')
    plt.title('最適α値の分布')
    plt.axvline(np.mean(alphas), color='red', linestyle='--', label=f'平均: {np.mean(alphas):.3f}')
    plt.legend()
    
    # βの分布
    plt.subplot(2, 2, 3)
    plt.hist(betas, bins=10, alpha=0.7, edgecolor='black')
    plt.xlabel('β (確率閾値)')
    plt.ylabel('頻度')
    plt.title('最適β値の分布')
    plt.axvline(np.mean(betas), color='red', linestyle='--', label=f'平均: {np.mean(betas):.3f}')
    plt.legend()
    
    # パラメータとROIの関係
    plt.subplot(2, 2, 4)
    # αとROIの関係
    plt.scatter(alphas, test_rois, alpha=0.7, label='α vs ROI', s=60)
    # βとROIの関係（右軸）
    ax2 = plt.gca().twinx()
    ax2.scatter(betas, test_rois, alpha=0.7, color='orange', label='β vs ROI', s=60, marker='^')
    
    plt.xlabel('パラメータ値')
    plt.ylabel('テストROI', color='blue')
    ax2.set_ylabel('テストROI', color='orange')
    plt.title('パラメータとROIの関係')
    plt.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    plt.savefig('outputs/optimal_parameters_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 統計サマリー
    logger.info("\n=== 最適パラメータ統計 ===")
    logger.info(f"α (期待値閾値) - 平均: {np.mean(alphas):.3f}, 標準偏差: {np.std(alphas):.3f}")
    logger.info(f"α範囲: {np.min(alphas):.3f} ~ {np.max(alphas):.3f}")
    logger.info(f"β (確率閾値) - 平均: {np.mean(betas):.3f}, 標準偏差: {np.std(betas):.3f}")
    logger.info(f"β範囲: {np.min(betas):.3f} ~ {np.max(betas):.3f}")

# 最適パラメータ分析実行
if optimization_results:
    analyze_optimal_parameters(optimization_results)